<a href="https://colab.research.google.com/github/BLOSSOM1994/calibration_wand/blob/main/MyWork/find_P_by_point_1_and_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [224]:
from PIL import Image
import numpy as np
import pandas as pd
import cv2

#from src.dlt import calculate_transformation_matrix
#from src.util import *

In [225]:

def homography_2d(x, y, u, v):
    h = np.array([
        [x[0], y[0], 1, 0, 0, 0, -u[0] * x[0], - u[0] * y[0]],
        [0, 0, 0, x[0], y[0], 1, -v[0] * x[0], - v[0] * y[0]],
        [x[1], y[1], 1, 0, 0, 0, -u[1] * x[1], - u[1] * y[1]],
        [0, 0, 0, x[1], y[1], 1, -v[1] * x[1], - v[1] * y[1]],
        [x[2], y[2], 1, 0, 0, 0, -u[2] * x[2], - u[2] * y[2]],
        [0, 0, 0, x[2], y[2], 1, -v[2] * x[2], - v[2] * y[2]],
        [x[3], y[3], 1, 0, 0, 0, -u[3] * x[3], - u[3] * y[3]],
        [0, 0, 0, x[3], y[3], 1, -v[3] * x[3], - v[3] * y[3]],
    ])
    b = np.array([u[0], v[0], u[1], v[1], u[2], v[2], u[3], v[3]])

    U, sigma, VT = np.linalg.svd(h)
    Sigma_pinv = np.zeros(h.shape).T
    Sigma_pinv[:len(sigma), :len(sigma)] = np.diag(1 / sigma[:])

    m = VT.T.dot(Sigma_pinv).dot(U.T).dot(b)
    p = np.array([[m[0], m[1], m[2]], [m[3], m[4], m[5]], [m[6], m[7], 1]])

    return p

In [226]:
def homography_3d(x, y, z, u, v):
    h = []
    for i in range(len(x)):
      h.append([x[i], y[i], z[i], 1, 0, 0, 0, 0, -u[i] * x[i], -u[i] * y[i], -u[i] * z[i]])
      h.append([0, 0, 0, 0, x[i], y[i], z[i], 1, -v[i] * x[i], -v[i] * y[i], -v[i] * z[i]])

    h = np.array(h)
    b = []
    for i in range(len(x)):
      b.append(u[i])
      b.append(v[i])

    b = np.array(b)

    U, sigma, VT = np.linalg.svd(h)

    Sigma_pinv = np.zeros(h.shape).T
    Sigma_pinv[:len(sigma), :len(sigma)] = np.diag(1 / sigma[:])

    m = VT.T.dot(Sigma_pinv).dot(U.T).dot(b)

    p = np.array([[m[0], m[1], m[2], m[3]], [m[4], m[5], m[6], m[7]], [m[8], m[9], m[10], 1]])

    return p

In [227]:
def calculate_transformation_matrix(pts_src, pts_dst):
    # There must be the same number of world points and pixel points
    assert (len(pts_dst) == len(pts_src))

    if len(pts_dst[0]) == 2:
        # At least 4 points are needed to calibrate a plane-to-plane projection
        assert (len(pts_dst >= 4))
        x = pts_dst[:, 0]
        y = pts_dst[:, 1]
        u = pts_src[:, 0]
        v = pts_src[:, 1]
        return homography_2d(x, y, u, v)


    elif len(pts_dst[0]) == 3:
        # At least 6 points to calibrate a 3D projection
        assert (len(pts_dst >= 6))
        x = pts_dst[:, 0]
        y = pts_dst[:, 1]
        z = pts_dst[:, 2]
        u = pts_src[:, 0]
        v = pts_src[:, 1]
        return homography_3d(x, y, z, u, v)

In [228]:
image_points=open('camera1.txt',"r")
#image_points=open('camera2.txt',"r")
#image_points=open('camera3.txt',"r")
world_point=open('mokhtasatvaghei.txt',"r")

In [229]:
y=0
cam_img=[]
for line in image_points:
  arr=[]
  image_coord= line.split()
  #print(image_coord)
  i=0
  for x in image_coord:
    arr.append(x)
   # print(x)
    i=i+1
  cam_img.append(arr)
  y=y+1

In [230]:
y=0
real_img=[]
for line in world_point:
  arr=[]
  world_coord= line.split()
  #print(image_coord)
  i=0
  for x in world_coord:
    arr.append(x)
   # print(x)
    i=i+1
  arr.pop(3)
  real_img.append(arr)
  y=y+1

In [231]:
# Calculate transformation matrix
cam_img=np.array(cam_img).astype(np.float)
real_img=np.array(real_img).astype(np.float)

matrix = calculate_transformation_matrix(cam_img, real_img)
matrix=str(matrix)

file_name_img= "cam1.txt"
#file_name_img= "cam2.txt"
#file_name_img= "cam3.txt"
file_img = open(file_name_img,"w+")
file_img.write("    p :\n \t([[m[0], m[1], m[2], m[3]], \n\t  [m[4], m[5], m[6], m[7]],\n\t  [m[8], m[9], m[10], 1]])")
file_img.write("\n.........................\nfor ")
file_img.write(image_points.name)
file_img.write("\n")
file_img.write(matrix)
file_img.close()